## Part A – Adaptive Diplomatic Scheduling (CSP) ##
## Implementation Plan
# We'll:

    Define the CSP with variables, domains, and constraints.
    Implement AC-3 for initial arc consistency.
    Use backtracking search with MRV, degree heuristic, and forward checking.
    Output the schedule in the required 3D format.
    Add a rescheduling module for when a room is destroyed.

In [1]:
import copy
from itertools import combinations

# Define constants
factions = [0, 1, 2, 3, 4, 5]  # 0:A, 1:B, 2:C, 3:D, 4:E, 5:F
rooms = [0, 1, 2]  # 0:R0 (neutral), 1:R1, 2:R2
variables = list(range(9))  # Time slots: 0 to 8 (3 per day)
pairs = list(combinations(factions, 2))  # All possible faction pairs

# Rivalry check: Group 0 = {0,1,2}, Group 1 = {3,4,5}
def is_rival(pair):
    return (pair[0] < 3 and pair[1] >= 3) or (pair[0] >= 3 and pair[1] < 3)

# Domain: (room, pair) with rivals restricted to R0
domain = [(room, pair) for pair in pairs for room in ([0] if is_rival(pair) else rooms)]

# Constraint functions
def diff_pair(val1, val2):
    """Ensure all pairs are unique across sessions."""
    return val1[1] != val2[1]

# Define CSP
csp = {
    'variables': variables,
    'domains': {var: domain.copy() for var in variables},
    'constraints': [(v1, v2, diff_pair) for v1 in variables for v2 in variables if v1 < v2]  # Unique pairs
}

# AC-3 for domain consistency
def revise(domains, var1, var2, constraint_func):
    revised = False
    to_remove = []
    for val1 in domains[var1]:
        if not any(constraint_func(val1, val2) for val2 in domains[var2]):
            to_remove.append(val1)
            revised = revised or True
    for val in to_remove:
        domains[var1].remove(val)
    return revised

def ac3(csp):
    domains = csp['domains']
    arcs = [(v1, v2, f) for v1, v2, f in csp['constraints']] + [(v2, v1, f) for v1, v2, f in csp['constraints']]
    queue = arcs.copy()
    while queue:
        v1, v2, f = queue.pop(0)
        if revise(domains, v1, v2, f):
            if not domains[v1]:
                return False
            queue.extend((v, v1, cf) for v, u, cf in arcs if v == v1 and u != v2)
    return True

# Forward checking
def forward_check(var, value, assignment, csp):
    domains = csp['domains']
    for v1, v2, f in csp['constraints']:
        if v1 == var and v2 not in assignment:
            to_remove = [val2 for val2 in domains[v2] if not f(value, val2)]
            for val2 in to_remove:
                domains[v2].remove(val2)
            if not domains[v2]:
                return False
        elif v2 == var and v1 not in assignment:
            to_remove = [val1 for val1 in domains[v1] if not f(val1, value)]
            for val1 in to_remove:
                domains[v1].remove(val1)
            if not domains[v1]:
                return False
    return True

# Backtracking with room consecutivity check
def backtrack(assignment, csp, faction_rooms):
    if len(assignment) == len(csp['variables']):
        return assignment
    var = min([v for v in csp['variables'] if v not in assignment])  # Static order
    day = var // 3
    for value in csp['domains'][var]:
        room, pair = value
        f1, f2 = pair
        # Check room consecutivity
        if day > 0 and ((faction_rooms[f1][day - 1] is not None and faction_rooms[f1][day - 1] == room) or
                        (faction_rooms[f2][day - 1] is not None and faction_rooms[f2][day - 1] == room)):
            continue
        assignment[var] = value
        faction_rooms[f1][day] = room
        faction_rooms[f2][day] = room
        if forward_check(var, value, assignment, csp):
            result = backtrack(assignment, csp, faction_rooms)
            if result is not None:
                return result
        del assignment[var]
        faction_rooms[f1][day] = None
        faction_rooms[f2][day] = None
    return None

# Reschedule after room destruction
def reschedule(csp, assignment, destroyed_room):
    # Update domains to exclude destroyed room
    for var in csp['domains']:
        csp['domains'][var] = [val for val in csp['domains'][var] if val[0] != destroyed_room]
    # Identify affected variables
    affected_vars = [var for var in assignment if assignment[var][0] == destroyed_room]
    # Remove affected assignments
    for var in affected_vars:
        del assignment[var]
    # Recompute faction_rooms from partial assignment
    faction_rooms = [[None] * 3 for _ in factions]
    for var in assignment:
        day = var // 3
        room, pair = assignment[var]
        f1, f2 = pair
        faction_rooms[f1][day] = room
        faction_rooms[f2][day] = room
    # Resume backtracking
    return backtrack(assignment, csp, faction_rooms)

# Pretty print schedule
def print_schedule(assignment, title="Meeting Schedule"):
    faction_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F'}
    room_map = {0: 'R0', 1: 'R1', 2: 'R2'}
    print(f"\n{title}:")
    for var in variables:
        day = var // 3
        slot = var % 3
        room, pair = assignment[var]
        f1, f2 = pair
        print(f"Day {day}, Slot {slot}: Room {room_map[room]}, Factions {faction_map[f1]} and {faction_map[f2]}")

# Main execution
print("Generating initial schedule with 9 time slots...")
csp_copy = copy.deepcopy(csp)
if ac3(csp_copy):
    faction_rooms = [[None] * 3 for _ in factions]
    assignment = backtrack({}, csp_copy, faction_rooms)
    if assignment:
        print_schedule(assignment, "Initial Schedule")
        # Simulate room destruction (Room R1)
        destroyed_room = 1
        print(f"\nRoom R{destroyed_room} destroyed. Rescheduling...")
        new_assignment = reschedule(copy.deepcopy(csp), assignment.copy(), destroyed_room)
        if new_assignment:
            print_schedule(new_assignment, "Rescheduled After Destruction")
        else:
            print("No solution found after room destruction.")
    else:
        print("No initial solution found.")
else:
    print("CSP unsatisfiable after AC-3.")

Generating initial schedule with 9 time slots...

Initial Schedule:
Day 0, Slot 0: Room R0, Factions A and B
Day 0, Slot 1: Room R0, Factions A and C
Day 0, Slot 2: Room R0, Factions A and D
Day 1, Slot 0: Room R1, Factions B and C
Day 1, Slot 1: Room R1, Factions D and E
Day 1, Slot 2: Room R1, Factions D and F
Day 2, Slot 0: Room R0, Factions A and E
Day 2, Slot 1: Room R0, Factions A and F
Day 2, Slot 2: Room R0, Factions B and D

Room R1 destroyed. Rescheduling...

Rescheduled After Destruction:
Day 0, Slot 0: Room R0, Factions A and B
Day 0, Slot 1: Room R0, Factions A and C
Day 0, Slot 2: Room R0, Factions A and D
Day 1, Slot 0: Room R2, Factions A and B
Day 1, Slot 1: Room R2, Factions A and C
Day 1, Slot 2: Room R2, Factions B and C
Day 2, Slot 0: Room R0, Factions A and E
Day 2, Slot 1: Room R0, Factions A and F
Day 2, Slot 2: Room R0, Factions B and D


    ## tweaked answer to part A ##
## Domain Definition:
    Ensure rival pairs (factions from different groups) are restricted to the neutral room (R0) in the domain, keeping it clean and efficient.
## Constraints:
    Unique Pairs: Each pair of factions meets exactly once.
    Same-Day Break: No faction participates in both sessions on the same day.
    Room Consecutivity: No faction uses the same room on consecutive days, enforced dynamically during backtracking.
    Rescheduling: Optimize rescheduling by adjusting only the affected sessions when a room is destroyed, rather than restarting the entire search.
## Output:
    Present a clear, structured schedule with day, slot, room, and faction details.

In [11]:
import copy
import random
from itertools import combinations

# Define constants
factions = [0, 1, 2, 3, 4, 5]  # 0:A, 1:B, 2:C, 3:D, 4:E, 5:F
rooms = [0, 1, 2]  # 0:R0 (neutral), 1:R1, 2:R2
variables = list(range(9))  # Time slots: Day 0 (0,1,2), Day 1 (3,4,5), Day 2 (6,7,8)
pairs = list(combinations(factions, 2))  # All possible faction pairs

# Rivalry check: Group 0 = {0,1,2}, Group 1 = {3,4,5}
def is_rival(pair):
    return (pair[0] < 3 and pair[1] >= 3) or (pair[0] >= 3 and pair[1] < 3)

# Domain: (room, pair) where rivals use only R0
domain = [(room, pair) for pair in pairs for room in ([0] if is_rival(pair) else rooms)]

# Constraint functions
def diff_pair(val1, val2):
    """Ensure all pairs are unique across sessions."""
    return val1[1] != val2[1]

def no_common_faction(val1, val2):
    """Ensure no faction appears in both sessions on the same day."""
    return not set(val1[1]) & set(val2[1])

# Define CSP
unique_pairs_constraints = [(v1, v2, diff_pair) for v1 in variables for v2 in variables if v1 < v2]
same_day_break_constraints = [
    (0, 1, no_common_faction), (0, 2, no_common_faction), (1, 2, no_common_faction),
    (3, 4, no_common_faction), (3, 5, no_common_faction), (4, 5, no_common_faction),
    (6, 7, no_common_faction), (6, 8, no_common_faction), (7, 8, no_common_faction)
]
csp = {
    'variables': variables,
    'domains': {var: domain.copy() for var in variables},
    'constraints': unique_pairs_constraints + same_day_break_constraints
}

# AC-3 for domain consistency
def revise(domains, var1, var2, constraint_func):
    revised = False
    to_remove = []
    for val1 in domains[var1]:
        if not any(constraint_func(val1, val2) for val2 in domains[var2]):
            to_remove.append(val1)
            revised = True
    for val in to_remove:
        domains[var1].remove(val)
    return revised

def ac3(csp):
    domains = csp['domains']
    arcs = [(v1, v2, f) for v1, v2, f in csp['constraints']] + [(v2, v1, f) for v1, v2, f in csp['constraints']]
    queue = arcs.copy()
    while queue:
        v1, v2, f = queue.pop(0)
        if revise(domains, v1, v2, f):
            if not domains[v1]:
                return False
            queue.extend((v, v1, cf) for v, u, cf in arcs if v == v1 and u != v2)
    return True

# Forward checking
def forward_check(var, value, assignment, csp):
    domains = csp['domains']
    for v1, v2, f in csp['constraints']:
        if v1 == var and v2 not in assignment:
            to_remove = [val2 for val2 in domains[v2] if not f(value, val2)]
            for val2 in to_remove:
                domains[v2].remove(val2)
            if not domains[v2]:
                return False
        elif v2 == var and v1 not in assignment:
            to_remove = [val1 for val1 in domains[v1] if not f(val1, value)]
            for val1 in to_remove:
                domains[v1].remove(val1)
            if not domains[v1]:
                return False
    return True

# Backtracking with room consecutivity
def backtrack(assignment, csp, rooms_used):
    if len(assignment) == len(csp['variables']):
        return assignment
    var = min([v for v in csp['variables'] if v not in assignment])
    day = var // 3
    for value in csp['domains'][var]:
        room, pair = value
        f1, f2 = pair
        if day > 0 and (
            (f1 in rooms_used[day-1] and rooms_used[day-1][f1] == room) or
            (f2 in rooms_used[day-1] and rooms_used[day-1][f2] == room)
        ):
            continue
        assignment[var] = value
        rooms_used[day][f1] = room
        rooms_used[day][f2] = room
        # Save domains
        saved_domains = {v: csp['domains'][v].copy() for v in csp['variables'] if v not in assignment}
        if forward_check(var, value, assignment, csp):
            result = backtrack(assignment, csp, rooms_used)
            if result is not None:
                return result
        # Restore domains
        for v in saved_domains:
            csp['domains'][v] = saved_domains[v]
        del rooms_used[day][f1]
        del rooms_used[day][f2]
        del assignment[var]
    return None
# Reschedule after room destruction
def reschedule(csp, assignment, destroyed_room):
    affected_vars = [var for var, (room, _) in assignment.items() if room == destroyed_room]
    if not affected_vars:
        return assignment
    # Remove affected assignments
    for var in affected_vars:
        del assignment[var]
    # Update domains
    for var in csp['domains']:
        csp['domains'][var] = [val for val in csp['domains'][var] if val[0] != destroyed_room]
    # Re-run backtracking on affected part
    rooms_used = [{} for _ in range(3)]
    for var in csp['variables']:
        if var in assignment:
            room, pair = assignment[var]
            day = var // 3
            f1, f2 = pair
            rooms_used[day][f1] = room
            rooms_used[day][f2] = room
    new_assignment = backtrack(assignment, csp, rooms_used)
    return new_assignment

# Pretty print schedule
def print_schedule(assignment, title="Meeting Schedule"):
    faction_map = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F'}
    room_map = {0:'R0', 1:'R1', 2:'R2'}
    print(f"\n{title}:")
    for var in variables:
        day = var // 3
        slot = var % 3
        room, pair = assignment[var]
        f1, f2 = pair
        print(f"Day {day}, Slot {slot}: Room {room_map[room]}, Factions {faction_map[f1]} and {faction_map[f2]}")

# Main execution
print("Generating initial schedule with 9 time slots...")
csp_copy = copy.deepcopy(csp)  # Preserve original CSP
if ac3(csp_copy):
    rooms_used = [{} for _ in range(3)]
    assignment = backtrack({}, csp_copy, rooms_used)
    if assignment:
        print_schedule(assignment, "Initial Schedule")
        # Simulate room destruction
        destroyed_room = random.choice([1, 2])  # R0 stays as neutral
        print(f"\nRoom {destroyed_room} (R{destroyed_room}) destroyed. Rescheduling...")
        new_assignment = reschedule(copy.deepcopy(csp), assignment.copy(), destroyed_room)
        if new_assignment:
            print_schedule(new_assignment, "Rescheduled After Destruction")
        else:
            print("No solution found after room destruction.")
    else:
        print("No initial solution found.")
else:
    print("CSP unsatisfiable after AC-3.")

Generating initial schedule with 9 time slots...

Initial Schedule:
Day 0, Slot 0: Room R1, Factions A and B
Day 0, Slot 1: Room R0, Factions C and D
Day 0, Slot 2: Room R1, Factions E and F
Day 1, Slot 0: Room R2, Factions A and C
Day 1, Slot 1: Room R0, Factions B and E
Day 1, Slot 2: Room R2, Factions D and F
Day 2, Slot 0: Room R0, Factions A and F
Day 2, Slot 1: Room R1, Factions B and C
Day 2, Slot 2: Room R1, Factions D and E

Room 2 (R2) destroyed. Rescheduling...

Rescheduled After Destruction:
Day 0, Slot 0: Room R1, Factions A and B
Day 0, Slot 1: Room R0, Factions C and D
Day 0, Slot 2: Room R1, Factions E and F
Day 1, Slot 0: Room R0, Factions A and B
Day 1, Slot 1: Room R0, Factions B and E
Day 1, Slot 2: Room R0, Factions E and F
Day 2, Slot 0: Room R0, Factions A and F
Day 2, Slot 1: Room R1, Factions B and C
Day 2, Slot 2: Room R1, Factions D and E


# Part B

In [3]:
import time
import copy

# Define constants
ACTIONS = ['Offer', 'Threaten', 'Bluff', 'Concede', 'Delay']
FACTIONS = ['A', 'B', 'C', 'D', 'E', 'F']
ROOMS = ['R0', 'R1', 'R2']

# Sample data (could be dynamic in a real system)
resource_urgency = {'A': 0.8, 'B': 0.6, 'C': 0.7, 'D': 0.9, 'E': 0.5, 'F': 0.4}
fatigue = {'A': 0.2, 'B': 0.3, 'C': 0.1, 'D': 0.4, 'E': 0.2, 'F': 0.3}
room_bias = {'R0': 0, 'R1': 1, 'R2': -1}  # Positive favors faction1, negative favors faction2
history = {('A', 'B'): 0.5, ('A', 'C'): -0.3, ('B', 'C'): 0.2}  # Positive = trust, negative = rivalry

# Game state class
class NegotiationState:
    def __init__(self, faction1, faction2, room, depth=0):
        self.faction1 = faction1
        self.faction2 = faction2
        self.room = room
        self.depth = depth
        self.action_history = []

    def is_terminal(self):
        return self.depth >= 3  # Negotiations end after 3 rounds

    def get_possible_actions(self):
        return ACTIONS

    def take_action(self, action1, action2):
        new_state = copy.deepcopy(self)
        new_state.action_history.append((action1, action2))
        new_state.depth += 1
        return new_state

    def get_utility(self):
        """Calculate utility for faction1 (faction2 gets the negative in zero-sum)."""
        utility = 0
        for action1, action2 in self.action_history:
            base = self.calculate_round_utility(action1, action2)
            # Modify with factors
            urgency_mod = resource_urgency[self.faction1] - resource_urgency[self.faction2]
            fatigue_mod = fatigue[self.faction2] - fatigue[self.faction1]  # Less fatigue = advantage
            bias_mod = room_bias[self.room] * 0.1  # Scale room bias
            pair = (self.faction1, self.faction2)
            hist_mod = history.get(pair, 0) * 0.2 if pair in history else 0
            utility += base + urgency_mod + fatigue_mod + bias_mod + hist_mod
        return utility

    def calculate_round_utility(self, action1, action2):
        """Base utility based on action pairs."""
        if action1 == 'Offer' and action2 == 'Concede':
            return 1.0
        elif action1 == 'Threaten' and action2 == 'Concede':
            return 0.7
        elif action1 == 'Bluff' and action2 == 'Delay':
            return -0.5
        elif action1 == 'Concede':
            return -0.8
        elif action2 == 'Concede':
            return 0.8
        return 0.0  # Neutral outcome

# Minimax with Alpha-Beta Pruning
def minimax(state, alpha, beta, maximizing_player):
    if state.is_terminal():
        return state.get_utility(), None

    if maximizing_player:  # Faction1 maximizes
        max_eval = float('-inf')
        best_action = None
        for action1 in state.get_possible_actions():
            for action2 in state.get_possible_actions():  # Simulate opponent’s response
                new_state = state.take_action(action1, action2)
                eval, _ = minimax(new_state, alpha, beta, False)
                if eval > max_eval:
                    max_eval = eval
                    best_action = action1
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            if beta <= alpha:
                break
        return max_eval, best_action
    else:  # Faction2 minimizes
        min_eval = float('inf')
        best_action = None
        for action1 in state.get_possible_actions():  # Opponent’s action
            for action2 in state.get_possible_actions():
                new_state = state.take_action(action1, action2)
                eval, _ = minimax(new_state, alpha, beta, True)
                if eval < min_eval:
                    min_eval = eval
                    best_action = action2
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            if beta <= alpha:
                break
        return min_eval, best_action

# Simulation function
def simulate_negotiations():
    sessions = [
        ('A', 'B', 'R0'),
        ('C', 'D', 'R1'),
        ('E', 'F', 'R2')
    ]

    for faction1, faction2, room in sessions:
        state = NegotiationState(faction1, faction2, room)
        start_time = time.time()
        utility, action = minimax(state, float('-inf'), float('inf'), True)
        end_time = time.time()
        print(f"\nNegotiation between {faction1} and {faction2} in {room}:")
        print(f"  Best Action for {faction1}: {action}")
        print(f"  Utility for {faction1}: {utility:.2f}")
        print(f"  Time Taken: {end_time - start_time:.4f} seconds")

# Run the simulation
if __name__ == "__main__":
    simulate_negotiations()


Negotiation between A and B in R0:
  Best Action for A: Offer
  Utility for A: 2.40
  Time Taken: 0.0188 seconds

Negotiation between C and D in R1:
  Best Action for C: Offer
  Utility for C: 1.80
  Time Taken: 0.0198 seconds

Negotiation between E and F in R2:
  Best Action for E: Offer
  Utility for E: 1.50
  Time Taken: 0.0207 seconds
